In [1]:
import pandas as pd
pd.options.display.max_columns=99
pd.options.display.max_rows=99
import numpy as np

%matplotlib inline

def tab_df(df_file,n):
    df_array=[]
    with open (df_file) as f:
        l=0
        for line in f:
            row=((line.split('\t')))
            if len(row)==n:
                df_array.append(row)
                l+=1
    columns=df_array[0]
    df_array.pop(0)
    df=pd.DataFrame(df_array,columns=columns)
    print df.shape
    return df

In [32]:
path="../data/"
cvr_file=path+'CVR_LOBBY_DISCLOSURE_CD.TSV'
cvr=tab_df(cvr_file,52)
cvr_cut=cvr[['FILING_ID','AMEND_ID','FILER_NAML']].reset_index(drop=True)
cvr_cut.shape

(301286, 52)


(301286, 3)

In [39]:
sum_file=path+'SMRY_CD.TSV'
summ=tab_df(sum_file,9)
sum_cut=summ[summ.LINE_ITEM=='ABCD'][['FILING_ID','AMEND_ID','AMOUNT_A']].reset_index(drop=True)
sum_cut.shape

(8593623, 9)


(188211, 3)

In [22]:
fil_file=path+'FILER_FILINGS_CD.TSV'
fil=tab_df(fil_file,16)
fil_cut=fil[['FILING_ID','FILING_SEQUENCE','FILING_DATE']].reset_index()
fil_cut.shape

(1946168, 16)


(1946168, 4)

In [40]:
cvr_cut['FILING_ID']=pd.to_numeric(cvr_cut.FILING_ID)
fil_cut['FILING_ID']=pd.to_numeric(fil_cut.FILING_ID)
sum_cut['FILING_ID']=pd.to_numeric(sum_cut.FILING_ID)

In [41]:
cf=cvr_cut.merge(fil_cut,how='left',left_on=['FILING_ID','AMEND_ID'], right_on=['FILING_ID','FILING_SEQUENCE'])

In [42]:
cfs=cf.merge(sum_cut,how='left',on=['FILING_ID','AMEND_ID'])

In [45]:
cfs[cfs.FILING_ID==2096383]

,FILING_ID,AMEND_ID,FILER_NAML,index,FILING_SEQUENCE,FILING_DATE,AMOUNT_A
298927,2096383,0,INVESTMENT COMPANY INSTITUTE,1936482.0,0,10/26/2016 2:07:32 PM,0
299133,2096383,1,INVESTMENT COMPANY INSTITUTE,1946039.0,1,12/13/2016 10:24:45 AM,0


In [127]:
cvr['report_date']=pd.to_datetime(cvr.RPT_DATE,errors='coerce')

In [128]:
cvr['report_year']=cvr.report_date.dt.year

In [129]:
df=cvr[cvr.report_date.dt.year==2016]
df.shape

(23386, 54)

In [147]:
dm=df.merge(sum_cut,how='left',on=['FILING_ID','AMEND_ID'])
dm.shape

(23386, 62)

In [156]:
dm[dm.FILING_ID==2038285]

,FILING_ID,AMEND_ID,REC_TYPE_x,FORM_TYPE_x,SENDER_ID,FILER_ID,ENTITY_CD,FILER_NAML,FILER_NAMF,FILER_NAMT,FILER_NAMS,REPORT_NUM,RPT_DATE,FROM_DATE,THRU_DATE,CUM_BEG_DT,FIRM_ID,FIRM_NAME,FIRM_CITY,FIRM_ST,FIRM_ZIP4,FIRM_PHON,MAIL_CITY,MAIL_ST,MAIL_ZIP4,MAIL_PHON,SIG_DATE,SIG_LOC,SIG_NAML,SIG_NAMF,SIG_NAMT,SIG_NAMS,PRN_NAML,PRN_NAMF,PRN_NAMT,PRN_NAMS,SIG_TITLE,NOPART1_CB,NOPART2_CB,PART1_1_CB,PART1_2_CB,CTRIB_N_CB,CTRIB_Y_CB,LBY_ACTVTY,LOBBY_N_CB,LOBBY_Y_CB,MAJOR_NAML,MAJOR_NAMF,MAJOR_NAMT,MAJOR_NAMS,RCPCMTE_NM,RCPCMTE_ID,report_date,report_year,index,LINE_ITEM,REC_TYPE_y,FORM_TYPE_y,AMOUNT_A,AMOUNT_B,AMOUNT_C,ELEC_DT
11346,2038285,0,CVR,F635,E00352,E00352,LEM,CHEVRON CORPORATION AND ITS SUBSIDIARIES,,,,000,4/30/2016 12:00:00 AM,1/1/2016 12:00:00 AM,3/31/2016 12:00:00 AM,1/1/2015 12:00:00 AM,E00352,CHEVRON CORPORATION AND ITS SUBSIDIARIES,SAN RAMON,CA,94583-0716,(925) 790-6454,,,,,4/30/2016 12:00:00 AM,"SAN RAFAEL, CA",KAUNE,JASON D.,,,KAUNE,JASON D.,,,ATTORNEY & AGENT FOR FILER,,,,,X,,SEE ATTACHMENT 1,,,,,,,,\n,2016-04-30,2016.0,8161630.0,ABCD,SMRY,F635,797795.17,,,\n
21827,2038285,1,CVR,F635,E00352,E00352,LEM,CHEVRON CORPORATION AND ITS SUBSIDIARIES,,,,001,4/30/2016 12:00:00 AM,1/1/2016 12:00:00 AM,3/31/2016 12:00:00 AM,1/1/2015 12:00:00 AM,E00352,CHEVRON CORPORATION AND ITS SUBSIDIARIES,SAN RAMON,CA,94583-0716,(925) 790-6454,,,,,10/21/2016 12:00:00 AM,"SAN RAFAEL, CA",KAUNE,JASON D.,,,KAUNE,JASON D.,,,ATTORNEY & AGENT FOR FILER,,,,,X,,SEE ATTACHMENT 1,,,,,,,,\n,2016-04-30,2016.0,8591931.0,ABCD,SMRY,F635,802187.71,,,\n


In [131]:
dm=dm[~dm.PER_TOTAL.isnull()]
dm=dm[dm.PER_TOTAL>0]
dm.reset_index(inplace=True,drop=True)
dm.shape

(11336, 56)

In [132]:
dm_cut=dm[['FILING_ID','AMEND_ID','report_date','FILER_NAML','PER_TOTAL','CUM_TOTAL']]
dm_cut.shape

(11336, 6)

In [133]:
dm_cut.to_csv(path+"lobbying_reports.csv",index=False,date_format="%Y-%m-%d")
dm_cut.shape

(11336, 6)

In [134]:
#dm_cut.sort_values('report_date',ascending=False)